In [1]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/23 13:15:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/23 13:15:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5554)

In [5]:
lines = socket_stream.window( 20 )

In [6]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [9]:
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ))  # Registers to a table.

__________
### Now run TweetRead.py
__________

In [10]:
ssc.start()    

23/02/23 13:16:10 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to 127.0.0.1:5554
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:476)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:218)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:200)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:394)
	at java.net.Socket.connect(Socket.java:606)
	at java.net.Socket.connect(Socket.java:555)
	at java.net.Socket.<init>(Socket.java:451)
	at java.net.Socket.<init>(Socket.java:228)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.start(Recei

23/02/23 13:16:14 ERROR JobScheduler: Error running job streaming job 1677186970000 ms.1
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_30464/1799013702.py", line 7, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 675, in createDataFrame
    return self._create_dataframe(d

23/02/23 13:16:21 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/23 13:16:21 WARN BlockManager: Block input-0-1677186981600 replicated to only 0 peer(s) instead of 1 peers
23/02/23 13:16:22 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/23 13:16:22 WARN BlockManager: Block input-0-1677186982600 replicated to only 0 peer(s) instead of 1 peers
23/02/23 13:16:23 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/23 13:16:23 WARN BlockManager: Block input-0-1677186983600 replicated to only 0 peer(s) instead of 1 peers
23/02/23 13:16:24 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/23 13:16:24 WARN BlockManager: Block input-0-1677186984600 replicated to only 0 peer(s) instead of 1 peers
23/02/23 13:16:25 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/23 13:16:25 WARN BlockManager: Block input-0-1677186985600 replicated to

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [ ]:
count = 0
while count < 10:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    count += 1
print(top_10_df)

In [ ]:
count = 0
while count < 10:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    sns.plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=top_10_df)
    sns.plt.show()
    count = count + 1

In [28]:
ssc.stop()

23/02/23 13:15:17 WARN StreamingContext: StreamingContext has already been stopped
